<a href="https://colab.research.google.com/github/fadhluibnu/ANOMALY_IOT_NETWORK_DETECTION/blob/main/Copy_of_IOT_ANOMALI_DETECTION_GROX_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy.stats import zscore
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# Pemuatan Data
train_file = "/content/drive/MyDrive/SEMESTER 4/RTI/ANOMALI DETECTION/UNSW_NB15_training-set.csv"
test_file = "/content/drive/MyDrive/SEMESTER 4/RTI/ANOMALI DETECTION/UNSW_NB15_testing-set.csv"
train_data = pd.read_csv(train_file, on_bad_lines="skip", low_memory=False)
test_data = pd.read_csv(test_file, on_bad_lines="skip", low_memory=False)

# Eksplorasi Data
print("\n📊 Training Data Info:")
print(train_data.info())
print("\n📊 Test Data Info:")
print(test_data.info())
print("\nTraining Data Shape:", train_data.shape)
print("Test Data Shape:", test_data.shape)
print("\nMissing Values in Training Data:")
print(train_data.isnull().sum())
print("\nMissing Values in Test Data:")
print(test_data.isnull().sum())

# Penanganan Nilai Hilang
train_data = train_data.dropna(subset=['attack_cat'])
test_data = test_data.dropna(subset=['attack_cat'])
numerical_cols = train_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in ['id', 'label', 'attack_cat']]
for col in numerical_cols:
    train_data[col] = train_data[col].fillna(train_data[col].median())
    test_data[col] = test_data[col].fillna(train_data[col].median())

# Penanganan Duplikasi
train_data = train_data.drop_duplicates()
test_data = test_data.drop_duplicates()
print("\nAfter Removing Duplicates - Train Shape:", train_data.shape)
print("After Removing Duplicates - Test Shape:", test_data.shape)

# Encoding attack_cat
attack_mapping = {
    'Normal': 0, 'Generic': 1, 'Exploits': 2, 'Fuzzers': 3, 'DoS': 4,
    'Reconnaissance': 5, 'Analysis': 6, 'Backdoor': 7, 'Shellcode': 8, 'Worms': 9
}
train_data['attack_cat'] = train_data['attack_cat'].map(attack_mapping)
test_data['attack_cat'] = test_data['attack_cat'].map(attack_mapping)

# Encoding Fitur Kategorikal
categorical_cols = train_data.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_train = encoder.fit_transform(train_data[categorical_cols])
encoded_test = encoder.transform(test_data[categorical_cols])
encoded_train = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(categorical_cols))
encoded_test = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(categorical_cols))
train_data = train_data.drop(columns=categorical_cols).reset_index(drop=True)
test_data = test_data.drop(columns=categorical_cols).reset_index(drop=True)
train_data = pd.concat([train_data, encoded_train], axis=1)
test_data = pd.concat([test_data, encoded_test], axis=1)
print("\nShape after encoding - Train:", train_data.shape)
print("Shape after encoding - Test:", test_data.shape)

# Normalisasi
scaler = StandardScaler()
train_data[numerical_cols] = scaler.fit_transform(train_data[numerical_cols])
test_data[numerical_cols] = scaler.transform(test_data[numerical_cols])
print("\nMax values after normalization:", train_data[numerical_cols].max())
print("Min values after normalization:", train_data[numerical_cols].min())

# Penanganan Outlier dengan Clipping
for col in numerical_cols:
    lower_bound = train_data[col].quantile(0.05)
    upper_bound = train_data[col].quantile(0.95)
    train_data[col] = train_data[col].clip(lower=lower_bound, upper=upper_bound)
    test_data[col] = test_data[col].clip(lower=lower_bound, upper=upper_bound)
print("\nAfter Clipping Outliers - Train Shape:", train_data.shape)
print("After Clipping Outliers - Test Shape:", test_data.shape)

# Seleksi Fitur dengan PCA (atau HBA sebagai alternatif)
X_train_pca = train_data.drop(columns=['id', 'label', 'attack_cat'])
X_test_pca = test_data.drop(columns=['id', 'label', 'attack_cat'])
scaler = StandardScaler()
X_train_pca_scaled = scaler.fit_transform(X_train_pca)
X_test_pca_scaled = scaler.transform(X_test_pca)
pca = PCA(n_components=0.95)
X_train = pca.fit_transform(X_train_pca_scaled)
X_test = pca.transform(X_test_pca_scaled)
print(f"\nJumlah komponen utama setelah PCA: {X_train.shape[1]}")

# Penanganan Ketidakseimbangan dengan SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, train_data['attack_cat'])
y_test = test_data['attack_cat']
print("\nLabel Distribution After SMOTE:")
print(pd.Series(y_train).value_counts())

# Konversi ke Tensor
train_data = train_data.replace([np.inf, -np.inf], np.nan).dropna()
test_data = test_data.replace([np.inf, -np.inf], np.nan).dropna()
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.int64)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.int64)
print("\nNaN in X_train_tensor:", torch.isnan(X_train_tensor).any())
print("Inf in X_train_tensor:", torch.isinf(X_train_tensor).any())

# DataLoader
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
batch_size = 64
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
print("\nTraining DataLoader - Number of batches:", len(train_loader))
print("Test DataLoader - Number of batches:", len(test_loader))

# Tambah dimensi channel untuk CNN
X_train_tensor = X_train_tensor.unsqueeze(1)
X_test_tensor = X_test_tensor.unsqueeze(1)
print("\nX_train_tensor Shape After Unsqueeze:", X_train_tensor.shape)
print("X_test_tensor Shape After Unsqueeze:", X_test_tensor.shape)

# Cek GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using Device: {device}")
if device.type == "cuda":
    print(f"🔧 GPU Name: {torch.cuda.get_device_name(0)}")



📊 Training Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 82332 non-null  int64  
 1   dur                82332 non-null  float64
 2   proto              82332 non-null  object 
 3   service            82332 non-null  object 
 4   state              82332 non-null  object 
 5   spkts              82332 non-null  int64  
 6   dpkts              82332 non-null  int64  
 7   sbytes             82332 non-null  int64  
 8   dbytes             82332 non-null  int64  
 9   rate               82332 non-null  float64
 10  sttl               82332 non-null  int64  
 11  dttl               82332 non-null  int64  
 12  sload              82332 non-null  float64
 13  dload              82332 non-null  float64
 14  sloss              82332 non-null  int64  
 15  dloss              82332 non-null  int64  
 16 

# Implementasi Model

## CNN + DBN (Ensemble)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import time
import warnings
warnings.filterwarnings("ignore")

# Cek apakah tqdm tersedia
try:
    from tqdm import tqdm
    use_tqdm = True
except ImportError:
    use_tqdm = False
    print("tqdm tidak tersedia. Menggunakan mode quiet...")

# Function untuk konversi data ke tensor
def convert_to_tensor(data):
    """Convert different data types to PyTorch tensor."""
    if isinstance(data, torch.Tensor):
        return data

    # Jika pandas Series atau DataFrame
    if hasattr(data, 'values'):
        data = data.values

    # Jika numpy array atau list
    if isinstance(data, (np.ndarray, list)):
        if isinstance(data[0], (int, np.integer)):
            return torch.tensor(data, dtype=torch.long)
        else:
            return torch.tensor(data, dtype=torch.float32)

    # Jika tipe data lain, coba konversi
    try:
        return torch.tensor(data)
    except:
        raise TypeError(f"Cannot convert {type(data)} to torch.Tensor")

# Fungsi untuk mendapatkan jumlah kelas unik
def get_num_classes(y_data):
    """Get number of unique classes in target data."""
    # Convert to tensor if not already
    y_tensor = convert_to_tensor(y_data)

    # Check number of unique classes
    return len(torch.unique(y_tensor))

# Membuat data loaders
def create_data_loaders(X_train, y_train, X_test, y_test, batch_size=32):
    # Konversi semua data ke tensor
    X_train_tensor = convert_to_tensor(X_train)
    y_train_tensor = convert_to_tensor(y_train)
    X_test_tensor = convert_to_tensor(X_test)
    y_test_tensor = convert_to_tensor(y_test)

    # Split training data untuk validation set
    X_train_np = X_train_tensor.numpy()
    y_train_np = y_train_tensor.numpy()

    X_train_split, X_val, y_train_split, y_val = train_test_split(
        X_train_np, y_train_np, test_size=0.2, random_state=42,
        stratify=y_train_np if len(y_train_np.shape) == 1 else None)

    # Konversi kembali ke tensor
    X_train_tensor_split = torch.tensor(X_train_split, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_train_tensor_split = torch.tensor(y_train_split, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)

    # Buat dataset
    train_dataset = TensorDataset(X_train_tensor_split, y_train_tensor_split)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    # Buat data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, val_loader, test_loader, X_train_tensor

# CNN Model
class CNN_Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CNN_Model, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(256)
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.conv3 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(512)
        self.pool3 = nn.AdaptiveMaxPool1d(output_size=1)

        # Fully connected layers
        self.fc1 = nn.Linear(512 * 1, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)

        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)

        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# RBM Model
class RBM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(input_size, hidden_size) * 0.1)
        self.b = nn.Parameter(torch.zeros(hidden_size))
        self.c = nn.Parameter(torch.zeros(input_size))

        self.input_size = input_size
        self.hidden_size = hidden_size

    def forward(self, v):
        batch_size = v.size(0)

        # Reshape input ke 2D jika perlu
        if v.dim() > 2:
            v = v.view(batch_size, -1)

        # Pra-proses input untuk memastikan dimensi yang benar
        if v.size(1) != self.input_size:
            v_resized = torch.zeros(batch_size, self.input_size, device=v.device)
            min_size = min(v.size(1), self.input_size)
            v_resized[:, :min_size] = v[:, :min_size]
            v = v_resized

        # Propagate visible to hidden
        h = torch.sigmoid(torch.matmul(v, self.W) + self.b)
        return h

    def reconstruct(self, h):
        # Reconstruct visible from hidden
        v_reconstructed = torch.sigmoid(torch.matmul(h, self.W.t()) + self.c)
        return v_reconstructed

# DBN Model
class DBN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(DBN, self).__init__()
        self.input_dim = input_dim

        # Two layers of RBM
        self.rbm1 = RBM(input_dim, hidden_dim1)
        self.rbm2 = RBM(hidden_dim1, hidden_dim2)

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_dim2, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        # Flatten input jika bukan 2D
        batch_size = x.size(0)
        if x.dim() > 2:
            x = x.view(batch_size, -1)

        h1 = self.rbm1(x)
        h2 = self.rbm2(h1)
        x = F.relu(self.fc1(h2))
        x = self.fc2(x)
        return x

# Ensemble (CNN + DBN)
class EnsembleModel(nn.Module):
    def __init__(self, cnn_model, dbn_model, output_dim):
        super(EnsembleModel, self).__init__()
        self.cnn_model = cnn_model
        self.dbn_model = dbn_model
        self.fc = nn.Linear(output_dim * 2, output_dim)  # Menggabungkan output CNN dan DBN

    def forward(self, x):
        # Save original input for DBN
        x_original = x.clone()

        # Input untuk CNN (tambahkan dimensi channel)
        x_cnn = x.unsqueeze(1)

        # Forward pass melalui CNN dan DBN
        cnn_output = self.cnn_model(x_cnn)
        dbn_output = self.dbn_model(x_original)

        # Gabungkan output CNN dan DBN
        combined_output = torch.cat((cnn_output, dbn_output), dim=1)

        # Final output
        final_output = self.fc(combined_output)

        return final_output

# Fungsi untuk visualisasi progres training
def plot_training_progress(train_losses, val_losses, train_accs, val_accs):
    plt.figure(figsize=(15, 5))

    # Plot losses
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)

    # Plot accuracies
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Training Accuracy')
    plt.plot(val_accs, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Fungsi untuk evaluasi model
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Calculate loss
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Return average loss and accuracy
    return running_loss / len(data_loader), 100 * correct / total

# Fungsi utama untuk training
def train_ensemble_model(ensemble_model, train_loader, val_loader, criterion, optimizer, num_epochs, device, verbose=0):
    # Lists untuk menyimpan metrics
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    best_val_acc = 0.0

    print(f"Training model for {num_epochs} epochs...")
    start_time = time.time()

    epoch_interate = 1

    for epoch in range(num_epochs):
        print(f"Start Epoch {1} : Loading...")
        epoch_interate += 1
        ensemble_model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop
        if use_tqdm and verbose > 0:
            loader = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        else:
            loader = train_loader

        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Reset gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = ensemble_model(inputs)

            # Calculate loss
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()

            # Update weights
            optimizer.step()

            # Statistik
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Hitung dan simpan metrik training
        epoch_train_loss = running_loss / len(train_loader)
        epoch_train_acc = 100 * correct / total
        train_losses.append(epoch_train_loss)
        train_accs.append(epoch_train_acc)

        # Evaluasi model pada validation set
        val_loss, val_acc = evaluate_model(ensemble_model, val_loader, criterion, device)
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        # Print statistik per epoch
        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        # Simpan model terbaik berdasarkan validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(ensemble_model.state_dict(), 'best_ensemble_model.pt')
            print(f"✅ Model saved with Val Acc: {val_acc:.2f}%")

    # Waktu training total
    total_time = time.time() - start_time
    print(f"Training completed in {total_time/60:.2f} minutes")

    # Plot and visualize training progress
    plot_training_progress(train_losses, val_losses, train_accs, val_accs)

    # Load best model
    try:
        ensemble_model.load_state_dict(torch.load('best_ensemble_model.pt'))
        print("Loaded best model based on validation accuracy")
    except:
        print("Couldn't load best model, using current model instead")

    return ensemble_model, train_losses, val_losses, train_accs, val_accs

# Fungsi untuk evaluasi dan visualisasi hasil
def evaluate_and_visualize(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    try:
        from sklearn.metrics import confusion_matrix, classification_report
        import seaborn as sns

        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        plt.show()

        # Classification report
        print("Classification Report:")
        print(classification_report(all_labels, all_preds))
    except ImportError:
        print("scikit-learn atau seaborn tidak tersedia untuk confusion matrix")

    # Accuracy
    accuracy = sum(1 for p, l in zip(all_preds, all_labels) if p == l) / len(all_preds)
    print(f"Test Accuracy: {accuracy*100:.2f}%")

    return accuracy

# Fungsi utama untuk melatih dan mengevaluasi model
def run_training(X_train, y_train, X_test, y_test, num_epochs=10, batch_size=32, verbose=0):
    print("CNN+DBN Ensemble Model Training")
    print("-" * 30)

    # Buat data loaders
    train_loader, val_loader, test_loader, X_train_tensor = create_data_loaders(
        X_train, y_train, X_test, y_test, batch_size)

    # Konversi y_train ke tensor untuk menghitung jumlah kelas
    y_train_tensor = convert_to_tensor(y_train)

    # Definisikan output_dim berdasarkan jumlah kelas
    output_dim = get_num_classes(y_train_tensor)

    # Input dimension
    input_dim = X_train_tensor.shape[1]
    print(f"Input dimension: {input_dim}, Output classes: {output_dim}")

    # Inisialisasi model
    cnn_model = CNN_Model(input_dim=input_dim, output_dim=output_dim)
    dbn_model = DBN(input_dim=input_dim, hidden_dim1=256, hidden_dim2=128, output_dim=output_dim)
    ensemble_model = EnsembleModel(cnn_model, dbn_model, output_dim=output_dim)

    # Setup device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    ensemble_model.to(device)

    # Loss dan optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(ensemble_model.parameters(), lr=0.001)

    # Test model dengan mini-batch
    if verbose > 0:
        print("\nTesting model with sample batch...")

    try:
        sample_inputs, sample_labels = next(iter(train_loader))
        sample_inputs, sample_labels = sample_inputs.to(device), sample_labels.to(device)

        with torch.no_grad():
            sample_outputs = ensemble_model(sample_inputs)

        if verbose > 0:
            print(f"Sample input shape: {sample_inputs.shape}")
            print(f"Sample output shape: {sample_outputs.shape}")
            print("Forward pass successful!")
    except Exception as e:
        print(f"Error during test: {e}")
        import traceback
        traceback.print_exc()
        return

    # Train model
    print("\nStarting model training...")
    ensemble_model, train_losses, val_losses, train_accs, val_accs = train_ensemble_model(
        ensemble_model, train_loader, val_loader, criterion, optimizer, num_epochs, device, verbose)

    # Evaluasi final
    print("\nPerforming final evaluation...")
    test_acc = evaluate_and_visualize(ensemble_model, test_loader, device)

    return ensemble_model, test_acc

# Gunakan fungsi ini untuk melatih model
if __name__ == "__main__":
    # Contoh penggunaan (ganti dengan data Anda sendiri)
    try:
        # verbose=0 untuk mengurangi output
        model, accuracy = run_training(X_train, y_train, X_test, y_test, num_epochs=10, verbose=0)
        print(f"Final accuracy: {accuracy:.2f}%")
    except NameError:
        print("X_train atau y_train tidak ditemukan.")
        print("Pastikan Anda telah mendefinisikan data sebelum menjalankan script ini.")

CNN+DBN Ensemble Model Training
------------------------------
Input dimension: 146, Output classes: 10
Using device: cpu

Starting model training...
Training model for 10 epochs...
Start Epoch 1 : Loading...
Epoch 1/10, Train Loss: 0.9031, Train Acc: 65.05%, Val Loss: 0.7518, Val Acc: 71.60%
✅ Model saved with Val Acc: 71.60%
Start Epoch 1 : Loading...
Epoch 2/10, Train Loss: 0.7265, Train Acc: 71.70%, Val Loss: 0.6857, Val Acc: 73.27%
✅ Model saved with Val Acc: 73.27%
Start Epoch 1 : Loading...
Epoch 3/10, Train Loss: 0.6594, Train Acc: 74.11%, Val Loss: 0.6356, Val Acc: 75.38%
✅ Model saved with Val Acc: 75.38%
Start Epoch 1 : Loading...
Epoch 4/10, Train Loss: 0.6220, Train Acc: 75.57%, Val Loss: 0.5934, Val Acc: 76.60%
✅ Model saved with Val Acc: 76.60%
Start Epoch 1 : Loading...
Epoch 5/10, Train Loss: 0.5964, Train Acc: 76.53%, Val Loss: 0.6149, Val Acc: 75.88%
Start Epoch 1 : Loading...
Epoch 6/10, Train Loss: 0.5753, Train Acc: 77.31%, Val Loss: 0.5559, Val Acc: 78.08%
✅ Mode

KeyboardInterrupt: 